# Graph Magma Class 

In [2]:
class graphMagma:
    from sage.matrix.constructor import block_matrix, zero_matrix, identity_matrix, matrix
    from sage.rings.integer_ring import ZZ
    #Create a global variable to store the generators
    generators = []
    Inc = matrix(ZZ, 0, 0, [])
    multiplication_table = matrix(SR,0,0,[])
    #Create a global variable to store the graph
    graph = []
    def __init__(self, graph:list):
        self.graph = graph
        self.generators = self.getGenerators()
        self.Inc = self.getIncidenceMatrix()
        self.multiplication_table = self.getMultiplicationTable()
    def getGenerators(self):
        self.generators = []
        for i in range(len(self.graph)):
            for j in range(self.graph[i][1]):
                self.generators.append(var(f"a_{i}_{j}", latex_name=fr"\alpha_{{{i},{j}}}"))
        return self.generators
    def getGraph(self):
        return self.graph
    def incidenceMatrix(self):
        G=self.graph
        I=self.Inc
        for g in G:
            if g[0] == 'K':
                I = block_matrix([[I, matrix(I.nrows(), g[1],[1]*(I.nrows()*g[1]))], [zero_matrix(g[1], I.ncols()), matrix(g[1], g[1], [1]*(g[1]*g[1]))]])
            else:
                I = block_matrix([[I, matrix(I.nrows(), g[1],[1]*(I.nrows()*g[1]))], [zero_matrix(g[1], I.ncols()), matrix(g[1],g[1])]])  
        self.Inc = I
    def getIncidenceMatrix(self):
        self.incidenceMatrix()
        return self.Inc  
    def multiplicationTable(self):
        I=self.Inc
        M=matrix(SR,I.nrows(),I.nrows())
        self.getGenerators()
        for i in range(I.nrows()):
            for j in range(I.nrows()):
                if I[i,j]==0:
                    M[i,j]=self.generators[j]
                else:
                    M[i,j]=self.generators[i]
        self.multiplication_table = M
    def getMultiplicationTable(self):
        self.multiplicationTable()
        return self.multiplication_table

# Graph Magma Algebra Class

In [169]:
from sage.algebras.free_algebra import FreeAlgebra
from sage.algebras.free_algebra_quotient import FreeAlgebraQuotient
from sage.structure.element import Element
from random import choice
relations=[]
class GraphMagmaAlg:
    """
    This class implements the graph magma algebra.
    It is a quotient of the free algebra over the base ring.
    The generators are the elements of the graph.
    The relations are given by the incidence matrix of the graph.

    The attributes of the class are:
    - base_ring: the base ring of the algebra
    - generators: the generators of the algebra
    - graph: the graph of the algebra
    - multiplication_table: the multiplication table of the algebra
    - magma_algebra: the quotient of the free algebra
    - gens: the generators of the quotient algebra
    - local_decomposition: the generators of cyclic left local projective ideals in the indecomposable decomposition of the graph magma algebra
    - simple_left_ideals: the simple left ideals of local projective components of the graph magma algebra
    - relations: the relations of the algebra induced by the incidence matrix of the graph
    - Indecomposable_Injective: the indecomposable injective left modules over the graph magma algebra indexed by the local projective components of the graph
    """
    def __init__(self,base_ring,GM:graphMagma):
        R=FreeAlgebraQuotient
        self.base_ring=base_ring
        self.generators=GM.getGenerators()
        self.generators_str=[str(g) for g in self.generators]
        self.graph=GM.getGraph()
        self.multiplication_table=GM.getMultiplicationTable()
        # Determine the number of generators from the list
        n = len(self.generators)
        # Create a string for the generator names
        generator_names_str =', '.join(self.generators_str)
        #g_str=f"R.<{generator_names_str}>=FreeAlgebraQuotient(A,self.mons,self.mats)";print(g_str)
        # Create the FreeAlgebra instance
        A = FreeAlgebra(base_ring, n,generator_names_str)
        self.alg=A
        F=A.monoid() #generator_names_str=F.gens();print(F.gens())
        self.mons=[F(1)]+list(F.gens())
        self.mats=self.build_relations()
        self.magma_algebra=FreeAlgebraQuotient(A, self.mons, self.mats, names=self.generators_str)
        splt=[g[1] for g in self.graph]
        L=self.magma_algebra.gens()
        self.gens=self.split_list(list(L),list(splt))
        self.local_decomposition=self.getDecomposition()
        self.simple_left_ideals=[self.getSimpleLeftIdeals(i) for i in range(len(self.graph)+1)]
        self.magma_algebra.generators=self.split_list(list(self.magma_algebra.gens()),list(splt))
    
    def __contains__(self, item):
        return item in self.magma_algebra
    
    def __str__(self):
        return f"GraphMagmaAlg with generators {list(self.generators_str)}"
    def __repr__(self):
        return f"GraphMagmaAlg with generators {list(self.generators_str)}"

    def build_relations(self):
        act=[];L=[];idx=0
        n=len(self.generators)
        self.relations=[];M=MatrixSpace(self.base_ring,n+1);I=identity_matrix(n+1)
        T=self.multiplication_table
        for j in range(T.nrows()):
            L=list(I[j+1])
            for i in range(T.ncols()):
                idx=self.generators.index(T[i,j])
                L+=list(I[idx+1])
            act.append(M(L))
        return act
            
    def split_list(self,L, lengths):
        result = []
        index = 0
        for length in lengths:
            sublist = L[index:index + length]
            result.append(sublist)
            index += length
        return result

    def getDecomposition(self):
        R=self.magma_algebra;g=self.gens
        L=[R(f[0]) for f in g]
        ideal_generators=[L[0]]+[L[i]-L[i-1] for i in [1..len(L)-1]]+[R(1)-L[-1]]
        return ideal_generators

    def getSimpleLeftIdeals(self,n:int):
        '''This function gives the generators of the simple left ideals in a particular local projective component.
        n represents the index of the local projective component.     '''
        R=self.magma_algebra;G=self.graph;L=self.gens
        if n==0:
            if G[0][0]=='N':
                return [L[0][0]]
            else:
                return [L[0][i]-L[0][0] for i in range(1,len(L[0]))]
        else:
            if n<len(G)-1:
                sub_graph=G[n-1:n+1]
                if sub_graph[0][0]=='K' and sub_graph[1][0]=='K': # (K,K)
                    if sub_graph[1][1]==1:
                        return [L[n][0]-L[n-1][0]]
                    else:
                        return [L[n][0]-L[n][i] for i in range(1,len(L[n]))]
                elif sub_graph[0][0]=='K' and sub_graph[1][0]=='N': # (K,N)
                    return [L[n][0]-L[n-1][0]]
                elif sub_graph[0][0]=="N" and sub_graph[1][0]=="K": # (N,K)
                    if sub_graph[0][1]==sub_graph[1][1]:
                        return [L[n][0]-L[n-1][0]]
                    elif sub_graph[0][1]>1 and sub_graph[1][1]==1:
                        return [L[n-1][i]-L[n-1][0] for i in range(1,len(L[n-1]))]
                    elif sub_graph[0][1]==1 and sub_graph[1][1]>1:
                        return [L[n][0]-L[n][i] for i in range(1,len(L[n]))] 
                    else:
                        return [L[n-1][i]-L[n-1][0] for i in range(1,len(L[n-1]))]+[L[n][0]-L[n][i] for i in range(1,len(L[n]))]
                elif sub_graph[0][0]=="N" and sub_graph[1][0]=="N": # (N,N)
                    if sub_graph[0][1]==1:
                        return [L[n][0]-L[n-1][0]]
                    else:
                        return [L[n-1][i]-L[n-1][0] for i in range(1,len(L[n-1]))]
            else:
                if G[n-1][0]=='N':
                    return [L[n-1][i]-L[n-1][0] for i in range(1,len(L[n-1]))]
                else:
                    return [1-L[n-1][0]]

    def Indecomposable_Injective(self,m:int,return_actions=False):
        R=self.magma_algebra;G=self.graph;L=R.generators;dim=0
        actions={}
        if m>0:
            if m<len(G)-1:
                sub_graph=G[m:m+2]
                if sub_graph[0][0]=='N' and sub_graph[1][0]=='N': # (N,N)
                    dim=sub_graph[1][1]
                    for l in L:
                        if L.index(l)<=m:
                            for g in l:
                                actions[str(g)]=matrix(QQ,dim,dim)
                        elif L.index(l)==m+1:
                            mat=matrix(QQ,dim,dim,[[1]+[0]*(dim-1)]+[[0]*dim for i in range(dim-1)])
                            for g in l:
                                actions[str(g)]=mat+self.shift_columns_right(mat,l.index(g))
                        else:
                            for g in l:
                                actions[str(g)]=identity_matrix(QQ,dim)
                    if return_actions:
                        return actions,GraphMagmaModule(self.magma_algebra,actions)
                    else:
                        return GraphMagmaModule(self.magma_algebra,actions)
                    return actions
                else:
                    return f"Under construction"
            else:
                return f"Under construction"
        else:
            return f"Under construction"

    def shift_columns_right(self,M, k):
        """
        Shifts the columns of matrix M to the right by k positions.
        Vacant positions on the left are filled with zeros.
    
        Args:
            M: A SageMath matrix.
            k: Number of positions to shift to the right.
    
        Returns:
            A new matrix with shifted columns.
        """
        if k==0:
            return matrix(M.base_ring(),M.nrows(),M.ncols())
        else:
            nrows, ncols = M.nrows(), M.ncols()
            k = k % (ncols + 1)  # Ensure k is within a valid range
            zero_part = Matrix(M.base_ring(), nrows, k)  # Create zero matrix of width k
            shifted_part = M[:, :ncols - k] if k < ncols else Matrix(M.base_ring(), nrows, 0)  # Remaining columns
            return zero_part.augment(shifted_part)

# GMA Module Class

In [162]:
from sage.modules.module import Module
from sage.modules.free_module import VectorSpace
from sage.rings.rational_field import QQ
from sage.matrix.constructor import matrix

class GraphMagmaModule(Module):
    def __init__(self, algebra, scalar_mults:dict):
        """
        algebra: graphMagmaAlgebra nesnesi
        dimension: Modülün boyutu (vektör uzayı boyutu)
        scalar_mults: graphMagmaAlgebra taban elemanlarının modül taban elemanları üzerindeki etkilerini temsil eden matrisler
        """
        self.algebra = algebra
        self.dimension = next(iter(scalar_mults.values())).nrows()  # Modülün boyutu
        self.vector_space = VectorSpace(QQ, self.dimension)  # Vektör uzayı
        self.scalar_mults = scalar_mults
        
        # Modülün taban elemanlarını oluştur
        self.basis = [self.vector_space.basis()[i] for i in range(self.dimension)]
    
    def act_on(self, algebra_element:str, vector):
        """
        Modül elemanına algebra elemanı ile çarpım etkisini uygular.
        algebra_element: graphMagmaAlgebra'nın bir elemanı
        vector: Modülün bir elemanı (bir vektör)
        """
        
        # Algebra elemanının vektör uzayı üzerindeki etkisini matris çarpımı olarak uygula
        transformed_vector = self.scalar_mults.get(algebra_element) * vector
        
        return transformed_vector
    
    def __repr__(self):
        return f"GraphMagmaModule over {self.algebra} with dimension {self.dimension}\nBasis: {self.basis}"


# Examples

## Define the graph magma algebra GM

In [170]:
M=graphMagma([('K',2),('N',2),('N',3)])
GM=GraphMagmaAlg(QQ,M)
R=GM.magma_algebra
R.generators

[[a_0_0, a_0_1], [a_1_0, a_1_1], [a_2_0, a_2_1, a_2_2]]

In [181]:
GM.simple_left_ideals

[[-a_0_0 + a_0_1],
 [-a_0_0 + a_1_0],
 [-a_1_0 + a_1_1],
 [-a_2_0 + a_2_1, -a_2_0 + a_2_2]]

## Define a module structure on GM

In [182]:
GS=R.generators;GS
actions={str(GS[0][0]):matrix(QQ,3), str(GS[0][1]):matrix(QQ,3), str(GS[1][0]):matrix(QQ,3), str(GS[1][1]):matrix(QQ,3), str(GS[2][0]):matrix(QQ,3,3,[1,0,0,0,0,0,0,0,0]),
         str(GS[2][1]):matrix(QQ,3,3,[1,1,0,0,0,0,0,0,0]),str(GS[2][2]):matrix(QQ,3,3,[1,0,1,0,0,0,0,0,0])}
GMM=GraphMagmaModule(GM,actions);GMM

GraphMagmaModule over GraphMagmaAlg with generators ['a_0_0', 'a_0_1', 'a_1_0', 'a_1_1', 'a_2_0', 'a_2_1', 'a_2_2'] with dimension 3
Basis: [(1, 0, 0), (0, 1, 0), (0, 0, 1)]

In [179]:
GM.Indecomposable_Injective(1,return_actions=True)

({'a_0_0': [0 0 0]
  [0 0 0]
  [0 0 0],
  'a_0_1': [0 0 0]
  [0 0 0]
  [0 0 0],
  'a_1_0': [0 0 0]
  [0 0 0]
  [0 0 0],
  'a_1_1': [0 0 0]
  [0 0 0]
  [0 0 0],
  'a_2_0': [1 0 0]
  [0 0 0]
  [0 0 0],
  'a_2_1': [1 1 0]
  [0 0 0]
  [0 0 0],
  'a_2_2': [1 0 1]
  [0 0 0]
  [0 0 0]},
 GraphMagmaModule over Free algebra quotient on 7 generators ('a_0_0', 'a_0_1', 'a_1_0', 'a_1_1', 'a_2_0', 'a_2_1', 'a_2_2') and dimension 8 over Rational Field with dimension 3
 Basis: [(1, 0, 0), (0, 1, 0), (0, 0, 1)])

In [180]:
GMM=GM.Indecomposable_Injective(1)
GMM.act_on('a_2_1',GMM.basis[1])

(1, 0, 0)

## Scalar multiplications on GMM

In [183]:
for i in range(len(GM.graph)):
    for j in range(GM.graph[i][1]):
        for k in range(len(GMM.basis)):
            print(str(R.generators[i][j]) + ' times ' + str(GMM.basis[k])+' = '+str(GMM.act_on(str(R.generators[i][j]),GMM.basis[k])))

a_0_0 times (1, 0, 0) = (0, 0, 0)
a_0_0 times (0, 1, 0) = (0, 0, 0)
a_0_0 times (0, 0, 1) = (0, 0, 0)
a_0_1 times (1, 0, 0) = (0, 0, 0)
a_0_1 times (0, 1, 0) = (0, 0, 0)
a_0_1 times (0, 0, 1) = (0, 0, 0)
a_1_0 times (1, 0, 0) = (0, 0, 0)
a_1_0 times (0, 1, 0) = (0, 0, 0)
a_1_0 times (0, 0, 1) = (0, 0, 0)
a_1_1 times (1, 0, 0) = (0, 0, 0)
a_1_1 times (0, 1, 0) = (0, 0, 0)
a_1_1 times (0, 0, 1) = (0, 0, 0)
a_2_0 times (1, 0, 0) = (1, 0, 0)
a_2_0 times (0, 1, 0) = (0, 0, 0)
a_2_0 times (0, 0, 1) = (0, 0, 0)
a_2_1 times (1, 0, 0) = (1, 0, 0)
a_2_1 times (0, 1, 0) = (1, 0, 0)
a_2_1 times (0, 0, 1) = (0, 0, 0)
a_2_2 times (1, 0, 0) = (1, 0, 0)
a_2_2 times (0, 1, 0) = (0, 0, 0)
a_2_2 times (0, 0, 1) = (1, 0, 0)
